3е задание

In [2]:
import gzip
import re


from dataclasses import dataclass
from typing import Iterator, List
from nltk.corpus import stopwords

@dataclass
class Text:
    label: str
    title: str
    text: str

# Чтение файла данных
def read_texts(fn: str="../data/news.txt.gz") -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

ru_stopwords = set(stopwords.words("russian"))

# Разбиение текста на слова                 
def tokenize_text(text: str) -> List[str]:
    text = text.lower()
    words = re.findall(r'\b\w+\b', text.lower())
    words = [w for w in words if w not in ru_stopwords]
    return words

# Текст без знаков припенания (нужен для gensim)
def normalize_text(text: str) -> str:
    return ' '.join(tokenize_text(text))

In [3]:
texts =[text for text in read_texts()]

In [4]:
sentences = [normalize_text(text.text) for text in texts]

Преобразовываем все новости вектора (384 размерности)

In [4]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm 
# Загрузка предобученной модели
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

vectors = []

# Создание векторных представлений для каждого текста
for sentence in tqdm(sentences):
    vector = model.encode([sentence])
    vectors.append(vector)



100%|██████████| 10000/10000 [19:09<00:00,  8.70it/s]


In [ ]:
import numpy as np

for i in range(0, 10000):
      np.savetxt("../vectorized_texts/" + str(i) + '.vec', vectors[i])

In [6]:
import os
import numpy as np
vectors = []
for file_name in os.listdir('../vectorized_texts'):
    if file_name.endswith('.vec'):
        vector = np.loadtxt(os.path.join('../vectorized_texts', file_name))
        vectors.append(vector)

Преобразовываем в Pandas DataFrame и добавляем нумерацию

In [7]:
import pandas as pd
vectors_for_table = vectors[:9500]
vects = []
for i, vect in enumerate(vectors_for_table):
    vects.append({"vector": vect, "num": i})
df = pd.DataFrame(vects)

In [8]:
df

,vector,num
0,"[-0.020230334252119064, 0.017395058646798134, ...",0
1,"[0.014233364723622799, 0.06375908106565475, -0...",1
2,"[-0.018548747524619102, 0.05791950970888138, -...",2
3,"[0.02038739062845707, 0.03758034110069275, 0.0...",3
4,"[0.0013368319487199187, 0.060717105865478516, ...",4
...,...,...
9495,"[-0.05632210150361061, 0.1072712317109108, -0....",9495
9496,"[-0.01065148413181305, 0.062431737780570984, -...",9496
9497,"[0.012071158736944199, 0.0795302465558052, -0....",9497
9498,"[0.018329815939068794, 0.038769543170928955, 0...",9498


Инициализируем таблицу lancedb

In [11]:
import lancedb

uri = "data/sample-lancedb"
db = lancedb.connect(uri)


tbl = db.create_table("my_table", data=df)

In [10]:
db.drop_table("my_table")

In [ ]:
vectors[9500]

Функция поиска n ближайших соседей

In [13]:
tbl.search(vectors[9500]).limit(2).to_list()

[{'vector': [-0.0687246099114418,
   0.08194753527641296,
   -0.06000540778040886,
   -0.04057786986231804,
   0.06194806471467018,
   -0.06454075872898102,
   -0.05890326574444771,
   0.018175750970840454,
   -0.013909577392041683,
   0.030452508479356766,
   -0.0363975428044796,
   -0.09439071267843246,
   -0.04899365454912186,
   -0.04396061971783638,
   -0.04447924718260765,
   0.007682339288294315,
   -0.07257269322872162,
   -0.0038053416647017,
   -0.10517581552267075,
   0.01786644384264946,
   0.020644303411245346,
   -0.12944866716861725,
   -0.012244593352079391,
   -0.08284163475036621,
   0.04392660781741142,
   -0.012779024429619312,
   0.005302719306200743,
   -0.03656194359064102,
   0.0036117639392614365,
   0.03268775716423988,
   -0.04454531893134117,
   0.007596693001687527,
   0.08469761162996292,
   0.03891122713685036,
   -0.01070426870137453,
   -0.022782061249017715,
   0.03359748050570488,
   -0.0714348554611206,
   -0.03813454881310463,
   -0.0134383505210280

In [14]:
texts[9500].text

'С января по май российские банки выдали рублевых ипотечных кредитов на 379,1 миллиарда рублей. По сравнению с аналогичным периодом прошлого года объем предоставленных займов снизился на 41,5 процента. Об этом свидетельствуют данные Центробанка.Всего россияне должны кредитным организациям 3,6 триллиона рублей, из них просроченная задолженность — 36,7 миллиарда (за год она выросла на 27,9 процента). Средняя ставка по ипотеке в рублях составила 13,5 процента.Кроме того, за пять месяцев банки выдали валютных жилищных кредитов на 2,3 миллиарда рублей (объем упал на 40 процентов). Всего россияне задолжали финансовым организациям 116,6 миллиарда рублей, 18,2 из них приходится на просроченную задолженность (рост на 24 процента). Средняя ставка — 11,3 процента.27 марта директор департамента финансовой стабильности ЦБ Сергей Моисеев сказал, что в 2015 году объем выдачи ипотечных кредитов упадет на 50 процентов, в 2016-м — вырастет на 10.Ранее, 13 марта, премьер-министр Дмитрий Медведев подписал

In [15]:
texts[7450].text

'Компания «Медиалогия» составила рейтинг цитируемости российских блогеров за 2014 год. По итогам года лидерами по частоте упоминания в СМИ стали Рамзан Кадыров, Дмитрий Рогозин и Алексей Навальный.Блоги Кадырова российские СМИ в этом году цитировали чаще других. Поэтому глава Чечни переместился со второго места на первое, потеснив прежнего лидера рейтинга Алексея Навального. Внимание журналистов привлекли записи Кадырова о ходе спецоперации в Доме печати в Грозном, сообщение об освобождении журналистов LifeNews Олега Сидякина и Марата Сайченко и новость о том, что один из лидеров «Исламского государства» Омар аш-Шишани убит.Улучшил свое положение в рейтинге и вице-премьер Дмитрий Рогозин, отмечавший День Победы в Приднестровье. Внимание журналистов привлекло сообщение Рогозина о том, что его самолет не выпускают из непризнанной республики. «Румыния по требованию США закрыла моему борту воздушное пространство, — написал вице-премьер в Twitter. — Украина снова не пропускает. В следующий 

Скрипты создания таблицы и поиска в таблице вынесены в отдельные скрипты, принимающие аргументы командной строки